# Load in the .ply file as a dataframe.

In [1]:
from plyfile import PlyData
import pandas as pd
from tqdm.notebook import tqdm

# Load the .ply file
ply = PlyData.read("bathtub.ply")

# Extract vertex data (assuming Gaussian splats are stored under 'vertex')
vertex_data = ply['vertex'].data  # Structured NumPy array

# Convert structured array to Pandas DataFrame
df = pd.DataFrame(vertex_data)

# Print first few rows to verify
df.head()

NUM_GAUSSIANS = df.shape[0]
NUM_GAUSSIANS

19688

# Extract covariance matrix

In [2]:
import numpy as np
import pandas as pd



def quaternion_to_rotation_matrix(w, x, y, z):
    """ Convert a quaternion (w, x, y, z) into a 3×3 rotation matrix. """
    return np.array([
        [1 - 2 * (y**2 + z**2), 2 * (x*y - w*z), 2 * (x*z + w*y)],
        [2 * (x*y + w*z), 1 - 2 * (x**2 + z**2), 2 * (y*z - w*x)],
        [2 * (x*z - w*y), 2 * (y*z + w*x), 1 - 2 * (x**2 + y**2)]
    ])



p_bar = tqdm(range(NUM_GAUSSIANS))
curr = 0
def compute_covariance_matrix(row):

    """ Compute the covariance matrix for a given row with quaternion and scale values. """
    # Extract quaternion and scale values
    w, x, y, z = row['rot_0'], row['rot_1'], row['rot_2'], row['rot_3']
    scale_0, scale_1, scale_2 = row['scale_0'], row['scale_1'], row['scale_2']

    # Compute rotation matrix
    R = quaternion_to_rotation_matrix(w, x, y, z)

    # Compute scale diagonal matrix (square of scale values)
    D = np.diag([scale_0**2, scale_1**2, scale_2**2])

    # Compute covariance matrix: Σ = R D R^T
    cov_matrix = R @ D @ R.T

    # Progress bar stuff
    global curr
    if curr % 100 == 0:
        p_bar.update(100)
    curr += 1
    p_bar.refresh()
    return cov_matrix
            
# Compute covariance matrices for each row
df['cov_matrix'] = df.apply(compute_covariance_matrix, axis=1)

# Display the first few results
df['cov_matrix']

  0%|          | 0/19688 [00:00<?, ?it/s]

0        [[14.270220302057256, 0.030039504859420002, -0...
1        [[14.034422023086114, 0.3942824939663506, 0.98...
2        [[12.205749334288944, -0.005383796275049768, -...
3        [[14.35949498287602, -0.6124241066330384, -0.2...
4        [[11.752327533769492, 0.0169083245297531, 0.15...
                               ...                        
19683    [[43.29410002374311, -1.5783025148603325, 8.87...
19684    [[13.16064708324406, -1.23490286241826, 0.8377...
19685    [[37.00068350932147, -0.5779855211218533, 7.20...
19686    [[65.70514331384507, 5.558583875339712, -22.01...
19687    [[65.91033729043927, -10.362485586139705, 17.0...
Name: cov_matrix, Length: 19688, dtype: object

# Calculate eigenvectors, eigenvalues and subsequently the normal for each gaussian

In [14]:
def quaternion_to_rotation_matrix(w, x, y, z):
    """Convert a quaternion (w, x, y, z) into a 3×3 rotation matrix.
    
    Args:
        w: Scalar/real component of quaternion
        x, y, z: Vector/imaginary components of quaternion
        
    Returns:
        3x3 rotation matrix as numpy array
    """
    return np.array([
        [1 - 2*y*y - 2*z*z,     2*x*y - 2*w*z,     2*x*z + 2*w*y],
        [    2*x*y + 2*w*z, 1 - 2*x*x - 2*z*z,     2*y*z - 2*w*x],
        [    2*x*z - 2*w*y,     2*y*z + 2*w*x, 1 - 2*x*x - 2*y*y]
    ])
    

p_bar = tqdm(range(NUM_GAUSSIANS))
curr = 0

def compute_normal_from_rotation(row):
    cov_matrix = row['cov_matrix']
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    normal = eigenvectors[:, 2]  # Smallest eigenvalue's eigenvector
    
    global curr
    if curr % 1000 == 0:
        p_bar.update(1000)
        p_bar.refresh()
    curr += 1
    
    return normal, eigenvectors[:, 0], eigenvectors[:, 1], eigenvectors[:, 2], eigenvalues[0], eigenvalues[1], eigenvalues[2]

column_filter = [
    'normal',
    'eigenvector_0',
    'eigenvector_1',
    'eigenvector_2',
    'eigenvalue_0',
    'eigenvalue_1',
    'eigenvalue_2'
]
df_new_cols = pd.DataFrame(df.apply(compute_normal_from_rotation, axis=1).tolist(), columns=column_filter)
df = pd.concat([df, df_new_cols], axis=1)  # Merge back into df
df_normals = df[column_filter + ['x', 'y', 'z']]
df_normals.to_csv('normals.csv')

  0%|          | 0/19688 [00:00<?, ?it/s]